# Getting Started With The Explorer API Client

## Authentication

The explorer API validates every request it receives by generating a cryptographic digest using the filters and a secret key associated with your account in Altmetric Explorer and comparing it to the value of the `digest` query parameter included with the request.  If the values are the same then a data response will be returned.  If not, then the API will respond with HTTP status `400 Bad Request`.

This means you need to calculate the digest using the same method as the API server does when you create a request.  The process is quite involved (see the [documentation](https://www.altmetric.com/explorer/documentation/api#authentication) for more details) but the API client handles this part for you as long as you supply your api key and api secret when you create a new `Client` object.

You can get you api key and secret at https://www.altmetric.com/explorer/settings.

You can use whatever you like to distribute secrets but, for the purposes of demonstrating how the API works, we are using [python-dotenv](https://pypi.org/project/python-dotenv/) and you must set it up with your own key and secret for the examples to work.

Simply create a file called `.env` in the same folder as this notebook and add the following lines using your own keys from https://www.altmetric.com/explorer/settings.

```
API_KEY=xxxxxxxx
API_SECRET=yyyyyyy
```

Go ahead and do it now and then run the cell below to load your keys into environment variables this notebook can access.

In [1]:
%load_ext dotenv
%dotenv

Now that `dotenv` has loaded your keys into the environment, you can assign them to Python variables.

In [2]:
import os

API_KEY = os.getenv('API_KEY')
API_SECRET = os.getenv('API_SECRET')

##  Using the API Client

First, import the `api` package and create a `Client` object using the API URL and your key and secret.

In [3]:
from altmetric.explorer.api import Client

api_client = Client('https://www.altmetric.com/explorer/api', API_KEY, API_SECRET)

### Sending a request

Now we can construct a query.  Here, we are using the [mention_sources](https://www.altmetric.com/explorer/documentation/api#mention-sources) as an example but they all work in the same way.

The simplest request you can send to the api would be to get all the mention sources for all research outputs.

```python
api_client.get_mention_sources()
```

This would return all the mention sources with no filters applied but that's not a very useful example becuase you will probably be more interested querying Explorer for a subset of the data it holds.  You do this by passing your query, and a few control variables, when you construct the request.  For instance, this (runnable) example returns all the mention sources of type `policy` or `blog` that have been active in the past 3 days. The results are ordered by `profile-type` and behind the scenes the data is fetched from the api server in batches of 100.

In [4]:
response = api_client.get_mention_sources(
    page_size=100,
    order='profile-type',
    mention_sources_types=['type:policy', 'type:blog'],
    timeframe='3d')

Before we move on, please be aware of the following rules for constructing queries:

* Parameters are passed using Python's [named parameter syntax](https://treyhunner.com/2018/04/keyword-arguments-in-python/)
* There are 3 optional parameters that control how the request is handled:
  - `order`: specify a field to order the results by.
  - `page_size`: behind the scenes, the [Explorer API returns data in a sequence of pages](https://www.altmetric.com/explorer/documentation/api#pagination) so that you don't need to pull down every result every time you run a query.  The API handles this for you but you can control the size of the pages by setting the `page_size` parameter when you submit a query.  It defaulst to 25.
  - `page_number`: set this value if you want to start fetching data at a specific page.  The default is the first page.
* All other parameters will be treated as filters to be applied to the data so you can find the results you are interested in
  - Parameters with a single value will be submitted as a single query parameter
  - Parameters with a list (or tuple, or set) will be submitted as separate array parameters in the request.
  - For example, in the request above, the filters will be `filter[mention_sources_types][]=type:policy&filter[mention_sources_types][]=type:blog&filter[timeframe]=3d`
* Technically, you could also supply your own `key` and `digest` as keyword arguments but this behaviour is not supported by the client and the results of doing so are undefined.

### Reading the response

The client takes the raw response from the API and presents it to you as a `Response` object that has accessor methods for common operations

You can get the HTTP status code:

In [5]:
print('INFO : the API responded with status:', response.status_code)


INFO : the API responded with status: 200


The data from the `meta` tag in the response:

In [6]:
meta = response.meta
print(f'INFO : got metadata:')
for key, value in meta.items():
    print(f'  {key} = {repr(value)}')

INFO : got metadata:
  status = 'ok'
  description = 'All research outputs sorted by Altmetric Attention Score mentioned in the past three days'
  total-results = 83
  total-pages = 1
  total-mentions = 431


And, of course, the results of the query.

In this case, note that `response.data` returns a Python [generator](https://realpython.com/introduction-to-python-generators/) so that you can pull the results from the api lazily instead of needing to download everything in one long-running query.  

A handy trick here is to use `islice` from Python's `itertools` library to pull down the first few rows so we can have a look at what we get.

In [7]:
from itertools import islice

data = response.data
for item in islice(data, 10):
    print(item)

{'id': 'blog:62498', 'type': 'profile', 'attributes': {'profile-type': 'blog', 'name': 'Balkinization', 'image': None}, 'meta': {'mention-count': 90}}
{'id': 'blog:79222', 'type': 'profile', 'attributes': {'profile-type': 'blog', 'name': 'information for practice - Journal Article Abstracts', 'image': None}, 'meta': {'mention-count': 58}}
{'id': 'blog:54632', 'type': 'profile', 'attributes': {'profile-type': 'blog', 'name': 'AWOL - The Ancient World Online', 'image': None}, 'meta': {'mention-count': 31}}
{'id': 'blog:83410', 'type': 'profile', 'attributes': {'profile-type': 'blog', 'name': 'news-medical.net', 'image': None}, 'meta': {'mention-count': 19}}
{'id': 'blog:76592', 'type': 'profile', 'attributes': {'profile-type': 'blog', 'name': 'Science Alert', 'image': None}, 'meta': {'mention-count': 18}}
{'id': 'blog:80146', 'type': 'profile', 'attributes': {'profile-type': 'blog', 'name': 'Whats New', 'image': None}, 'meta': {'mention-count': 11}}
{'id': 'blog:60593', 'type': 'profile'

(If you really want all the results in one go, you should run `data = list(response.data)`)

Finally, you also have access to the raw response if you need to dig any deeper.  Behind the scenes the client is using the [requests](https://pypi.org/project/requests/) HTTP client and the raw response held is the result of running `requests.get(api_url)`.  

For example, you might use this to see the url that was submitted.

In [8]:
from urllib.parse import unquote

print(unquote(response.raw_response.url))

https://www.altmetric.com/explorer/api/research_outputs/mention_sources?page[size]=100&filter[order]=profile-type&key=b5f9faabd368491692d4209087ffacae&digest=788862429f1f577faacfb54c9bc9fa33075a4d98&filter[mention_sources_types][]=type:policy&filter[mention_sources_types][]=type:blog&filter[timeframe]=3d
